In [3]:
import numpy as np
from collections import Counter

def entropy(data):
    return -sum((count / len(data)) * np.log2(count / len(data)) for count in Counter(data).values())

def information_gain(data, feature_index, target):
    total_entropy = entropy(target)
    values, counts = np.unique(data[:, feature_index], return_counts=True)
    weighted_entropy = sum((counts[i] / len(target)) * entropy(target[data[:, feature_index] == values[i]]) for i in range(len(values)))
    return total_entropy - weighted_entropy

def id3(data, target, features, depth=0, max_depth=None):
    if len(np.unique(target)) == 1 or len(features) == 0 or (max_depth and depth >= max_depth): 
        return Counter(target).most_common(1)[0][0]
    feature_gains = [information_gain(data, i, target) for i in range(len(features))]
    best_feature = features[np.argmax(feature_gains)]
    tree = {best_feature: {}}
    for value in np.unique(data[:, np.argmax(feature_gains)]):
        sub_data, sub_target = data[data[:, np.argmax(feature_gains)] == value], target[data[:, np.argmax(feature_gains)] == value]
        tree[best_feature][value] = id3(np.delete(sub_data, np.argmax(feature_gains), axis=1), sub_target, features[:np.argmax(feature_gains)] + features[np.argmax(feature_gains) + 1:], depth + 1, max_depth)
    return tree

def print_tree(tree, depth=0):
    if isinstance(tree, dict):
        for key, value in tree.items():
            print("  " * depth + str(key))
            print_tree(value, depth + 1)
    else:
        print("  " * depth + f"--> {tree}")

data = np.array([
    ['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']
])
feature_names = ['Outlook', 'Temperature', 'Humidity', 'Wind']
target = data[:, -1]
tree = id3(data[:, :-1], target, feature_names)
print_tree(tree)

Outlook
  Overcast
    --> Yes
  Rain
    Temperature
      Cool
        Wind
          Strong
            --> No
          Weak
            --> Yes
      Mild
        --> Yes
  Sunny
    Humidity
      High
        --> No
      Normal
        --> Yes
